#### Milestone 1
Implement pupil detection and tracking in Python in real time. 

Achieve measurement of gaze angle in Python. Mark down the location of eye corner and iris center and determine the gaze vector numerically. Test and compare the effectiveness of mapping with pure mathematical equations and mapping with statistical modelling. 

Implement Lucas Kanade Tracking method, comparing the result with KCF Tracker. Milestone 1 result should be a cursor on the screen predicting the location that the users are looking at.
#### Milestone 2
Achieve measurement of gaze angle in C++/Android. Process the frame captured by the front camera of the Android Tablet implementing with Android Studio . 

Apply the mapping model achieved in the Python part to map the gaze angle into location on the screen. 

Milestone 2 result should be a cursor on the computer screen predicting the location that the users are looking at. At this time, since the mapping of model is fixed, the tracking may only be effected for the model of the group members’.
#### Milestone 3
Android application should start with tutorial to help our application to calibrate tracking. Then it should keep track of where you’re looking at in real time fashion by drawing cursor on the screen. By this time, it should be a completed app that can work independently.
#### Demo 
Complete application supporting two different modes, heatmap and real-time predicting with cursor. Additional feature includes the app running as a background service.


In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import scipy.signal as sg
import scipy as sp
from mpl_toolkits import mplot3d
import ellipses as el
from matplotlib.patches import Ellipse
import sys
%matplotlib inline
# helper functions
# ellipse fitting function
def efit(data, maxpR, right=True):
    lsqe = el.LSqEllipse()
    lsqe.fit(data)
    center, width, height, phi = lsqe.parameters()
    vector = [(center[0]-maxpR[1]), (center[1]-maxpR[0])]
    vecmag = np.sqrt(vector[0]**2+vector[1]**1)
    return center, vector
# remove noise
def debubble(cpos, cneg, rpos, rneg, win_):
#     in# put: xcor of edge points
#     ncol: number of column
#     posfirst: posedge happens before negedge
    counts = np.bincount(cpos)
    pmode = np.argmax(counts)
    counts2 = np.bincount(cneg)
    nmode = np.argmax(counts2)
    ta1 = np.zeros(len(cpos)).astype('bool')
    ta2 = np.zeros(len(cneg)).astype('bool')
    print('truncation: ', pmode, '+-', win_)
    print('truncation: ', nmode, '+-', win_)
    for i in range(len(cpos)):
        ta1[i] = (cpos<(pmode+win_))[i] and (cpos>(pmode-win_))[i]
    for i in range(len(cneg)):
        ta2[i] = (cneg<(nmode+win_))[i] and (cneg>(nmode-win_))[i]
    return cpos[ta1], cneg[ta2], rpos[ta1], rneg[ta2]
# imgEye : list of 2d array, image
# frame: 2d array
# facecor: (2,)array, upper right cor of face frame
# eye: t*4 array, list of eyes' cor, under face frame
# parameters:
zoomout = 1 # 
ratio = .32
narrower = 0 
pns = 50
nns = 50
win = 6 # bilateral window size in pixel
print ('modules imported')

modules imported


In [2]:
#   faces eyes detection
def getEyes(frame):
    face_cascade = cv.CascadeClassifier('C:\\Users\\wangy\\Anaconda2\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier('C:\\Users\\wangy\\Anaconda2\\Lib\\site-packages\\cv2\\data\\haarcascade_eye.xml')
    if (eye_cascade.empty()==True or face_cascade.empty()==True):
        print ('Fail to load xml data')
        return []
    imgEye = []
    cnt = 0
    img = frame
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if(len(faces) != 1):
        print("Face dectection failure")
        return []
    for (x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        roi_gray = gray[y:y+h, x:x+w]
#         cv.imwrite('pic'+str(counter)+'.png', frame )
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            imgEye.append(cv.cvtColor(roi_color[ey+narrower:ey+(int)(ew), ex:ex+(int)(eh*zoomout)], cv.COLOR_BGR2GRAY))
            cv.putText(roi_color, str(cnt), (ex-20, ey-20), cv.FONT_HERSHEY_SIMPLEX, 3, (0,0,255),thickness=5)
            cnt += 1
    return imgEye, frame, eyes, faces[0,0:2]
print('function imported !')

function imported !


In [3]:
def showGaze(p1, p2, counter):
    imgEye[0] = cv.cvtColor(frame[p1[1]:p2[1], p1[0]:p2[0]], cv.COLOR_BGR2GRAY)
#     cv.imwrite('eye'+str(counter)+'.png', imgEye[0])

    cfR = np.array([[-1,-1,-1,1,1,1],[-1,-1,-1,-1,1,1],[-1,-1,-1,-1,-1,1],[1,1,1,1,1,1]])
    print ((imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)]).shape)
    foR = sg.convolve2d(imgEye[0][:,:(int)(imgEye[0].shape[1]*ratio)], cfR, mode='valid')
    maxpR = np.array([(int) (np.argmax(foR)/foR.shape[1]), np.argmax(foR)%foR.shape[1]])
    gradx = cv.Sobel(imgEye[0],cv.CV_64F,1,0,ksize=5)
    gradx = gradx[:,(int)(imgEye[0].shape[1]*ratio):]
    row, col = gradx.shape[0], gradx.shape[1]
    ranks = np.argsort(gradx.flatten())
    cargp = (ranks[-pns:] % col).astype('int')
    rargp = (ranks[-pns:] / col).astype('int')
    cargp += (int) ((imgEye[0].shape[1])*ratio)
    cargn = (ranks[0:nns] % col).astype('int')
    rargn = (ranks[0:nns] / col).astype('int')
    cargn += (int)((imgEye[0].shape[1])*ratio)
    cargp_, cargn_, rargp_, rargn_ = debubble(cargp, cargn, rargp, rargn, win)
    xcor = np.append(cargp_, cargn_)
    ycor = np.append(rargp_, rargn_)
    for X in xcor: 
        for Y in ycor:
            pointx = (int) (X) + p1[0]
            pointy = (int) (Y) + p1[1]
#             cv.circle(frame, (pointx, pointy), 1, (255,0,0)) 
    center, _ = efit ([xcor, ycor], maxpR)
    arre = (p1[0] + (int) (maxpR[1]), p1[1] + (int) (maxpR[0]))
    arrs = (p1[0] + (int) (center[1]), p1[1] + (int) (center[0]))
    cv.arrowedLine(frame, arrs, arre, (0,0,255), 2)
    for i in range(len(xcor)):
        cv.circle(frame, (p1[0] + xcor[i], p1[1] + ycor[i]), 1, (97,0,255)) # purple 
    cv.circle(frame, (arrs), 3, (0,255,0)) # green RGB
    cv.circle(frame, (arre), 3, (255,0,127)) # purple 
    print (maxpR[1], maxpR[0], center[0], center[1])
#     cv.imwrite('face'+str(counter)+'.png', frame )
    counter += 1
    vector = [maxpR[1]-center[1], maxpR[0]-center[0]]
    return frame, vector, counter
print('function imported !')

function imported !


In [4]:
def calibrate_draw(outcx_, outcy_, frame, currpt):
#     Python: cv2.line(img, pt1, pt2, color[, thickness[, lineType[, shift]]]) → None
    for i in range((int)(len(outcx_)/2)): # use first half of the outcx_to draw lines
#         cv.line(frame, (outcx_[i], 0), (outcx_[i], frame.shape[0]), (0,0,0))
#         cv.line(frame, (0, outcx_[i]), (frame.shape[1], outcx_[i]), (0,0,0))
#         Python: cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]]) → None
        cv.circle(frame, (outcx_[currpt], outcy_[currpt]), 8, (255,255,0), thickness = 4)
    return frame

In [ ]:
video = cv.VideoCapture(0)
counter = 0
if not video.isOpened():
    print('Could not open video!')
    sys.exit()
tracker = None
calpts = 9
roi = (0, 0, 0, 0)
predx = np.zeros(calpts)
predy = np.zeros(calpts)
# outcx = [0,0,0,0]
# outcy = [0,0,0,0]
c0x = 0
c1x = 0
c0y = 0
c1y = 0
calibrate_flag = 0
tracking_flag = -1
while True:
    key = cv.waitKey(1) & 0xff
    read_success, frame = video.read()
    fx10 = (int) (frame.shape[1]/10)
    fy10 = (int) (frame.shape[0]/10)
    fx2 = (int) (frame.shape[1]/2)
    fy2 = (int) (frame.shape[0]/2)

#     frame = np.fliplr(frame)
    if not read_success:
        print('Cannot read video file!')
        sys.exit()
    if key == 13:
        if tracking_flag == -1:
        # calibrate in order upperleft, upperright, bottomleft, bottomright
            outcx = [fx10 * 2, fx10 * 4, fx10 * 6, fx10 * 2, fx10 * 4, fx10 * 6, fx10 * 2, fx10 * 4, fx10 * 6 ]
            outcy = [fy10 * 2, fy10 * 2, fy10 * 2, fy10 * 4, fy10 * 4, fy10 * 4, fy10 * 6, fy10 * 6, fy10 * 6 ]

            imgEye, frame, eye, facecor = getEyes(frame)
            cv.imshow("ECE420 Lab7", frame)
            key = cv.waitKey(0) & 0xff
            if (key == 13):
                continue
            if (key-48 >= len(imgEye)): 
                print("Cannot find eye!")
                sys.exit()
            roi = eye[key-48]
            roi = tuple(roi + np.append(facecor, [0,0]))
            tracking_flag = 0
        if tracking_flag == 1:
            roi = (0, 0, 0, 0)
            tracker.clear()
            reye = None
            tracking_flag = -1
    elif key == 27:
        break
    start = cv.getTickCount()
    if tracking_flag == -1:
        cv.putText(frame, "Press ENTER to select ROI!", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    elif tracking_flag == 0:
        # Initialize KCF Tracker and Start Tracking
        # 1. Create a KCF Tracker
        # 2. Initialize KCF Tracker with grayscale image and ROI
        # 3. Modify tracking flag to start tracking
        # Your code starts here
        tracker = cv.TrackerMedianFlow_create()
        tracker.init(frame, roi)
        tracking_flag = 1
    else:
        # Update tracking result is succeed
        # If failed, print text "Tracking failure occurred!" at top left corner of the frame
        # Calculate and display "FPS@fps_value" at top right corner of the frame
        # Your code starts here
        (success, roi) = tracker.update(frame)
        if success:
            p1 = (int(roi[0]), int(roi[1]))
            p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
            frame, vector, counter = showGaze(p1,p2, counter)
#             if (calibrate_flag == -1):
#                 calibrate_flag += 1
#                 break
            if (calibrate_flag < calpts): # calibrate
                frame = calibrate_draw(outcx, outcy, frame, calibrate_flag)
                key = cv.waitKey(0) & 0xff
                if (key == 13):
                    cv.imshow("ECE420 Lab7", frame)
                    key = cv.waitKey(0) & 0xff
                    if (key == 13): 
                        print('record point', calibrate_flag)
                        predx[calibrate_flag] = vector[0]
                        predy[calibrate_flag] = vector[1]
                        calibrate_flag += 1
            elif (calibrate_flag == calpts):
                # linear regression outcome = c1 * income + c0 
                vandx = np.vander(predx, 2)[:, ::-1]
                vandy = np.vander(predy, 2)[:, ::-1]
                lrsolx,_,_,_ = np.linalg.lstsq(vandx, outcx)
                lrsoly,_,_,_ = np.linalg.lstsq(vandy, outcy)
                c0x = lrsolx[0]
                c1x = lrsolx[1]
                c0y = lrsoly[0]
                c1y = lrsoly[1]
                calibrate_flag += 1
                plt.figure()
                plt.subplot(121)
                plt.plot(predx, outcx,'ro')
                plt.plot(predx, c0x + predx * c1x)
                plt.title('linregress for x cors')
                plt.xlabel('predictor x')
                plt.ylabel('predictor y')
                plt.subplot(122)
                plt.plot(predy, outcy,'bo')
                plt.plot(predy, c0y + predy * c1y)
                plt.title('linregres for y cors')
                plt.savefig('linregress_xy.png', dpi=256)
            else:
                cv.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    #             cv.imwrite('eye'+str(counter)+'.png', frame[p1[1]:p2[1], p1[0]:p2[0]])
                cursorx = vector[0] * c1x + c0x
                cursory = vector[1] * c1y + c0y
                print('cursor at',cursorx,',',cursory)
                if(cursorx>frame.shape[1] or cursorx<0 or cursory>frame.shape[0] or cursory<0):
                    cv.putText(frame, "Cursor Out Of Frame", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
                cv.circle(frame, ((int) (cursorx), (int) (cursory)), 8, (255,0,0), thickness=4) # order of cor not sure
        else:
            cv.putText(frame,"Tracking failure occured!",(10,20),cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    fps = cv.getTickFrequency() / (cv.getTickCount() - start);
    cv.putText(frame, "FPS : " + str(int(fps)), (100, 20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)
    cv.imshow("ECE420 Lab7", frame)

(66, 21)
truncation:  43 +- 6
truncation:  24 +- 6
12 29 32.71629997756131 31.536713912728032
(66, 20)
truncation:  42 +- 6
truncation:  25 +- 6
10 31 32.41981640980684 32.07499996578574
record point 0
(59, 18)
truncation:  38 +- 6
truncation:  22 +- 6
10 26 29.18746774427102 28.8310210553253
(58, 18)
truncation:  38 +- 6
truncation:  21 +- 6
12 23 28.859389300480615 28.142190108172798
(58, 18)
truncation:  37 +- 6
truncation:  20 +- 6
10 26 28.226789223974123 27.97697544362991
record point 1
(57, 18)
truncation:  37 +- 6
truncation:  20 +- 6
11 24 28.364825078800276 27.72538489470472
(56, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 23 27.153531621592414 26.88331472873338
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
10 25 26.931379402446076 26.79507874148518
(55, 17)
truncation:  36 +- 6
truncation:  20 +- 6
11 23 27.231668979878677 26.215786285831413
record point 2
(55, 17)
truncation:  36 +- 6
truncation:  20 +- 6
11 24 27.267341149130292 26.607683711682363
(55, 17)
trunca

C:\Users\wangy\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\wangy\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.74435129498556 26.471551279533244
cursor at 321.0065205209536 , 222.7117675359841
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.71765280241458 26.12861304043249
cursor at 333.01235988636415 , 221.86462938153124
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.651274087980976 26.15906838555477
cursor at 331.94615629730094 , 219.75844524058238
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.678904975157554 26.138396130810683
cursor at 332.669866112891 , 220.6351681007376
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.631368651756677 26.05034321578592
cursor at 335.7524886755293 , 219.1268494880581
(56, 17)
truncation:  35 +- 6
truncation:  19 +- 6
11 22 26.6559849307355 26.27046581345888
cursor at 328.0462714998413 , 219.9079193963376
(55, 17)
truncation:  35 +- 6
truncation:  19 +- 6
10 24 26.794792914326653 26.426316043138964
cursor at 287.5813988032303 , 160.85264630661874
(55

(53, 17)
truncation:  33 +- 6
truncation:  18 +- 6
11 21 25.559099757735787 25.424211584415374
cursor at 357.6725749498064 , 216.8337710551673
(53, 17)
truncation:  33 +- 6
truncation:  18 +- 6
10 22 25.50381696026733 25.746925817328265
cursor at 311.36600194023083 , 183.3498461070966
(53, 16)
truncation:  33 +- 6
truncation:  18 +- 6
10 21 25.333234056572262 25.102882814536365
cursor at 333.91314295376026 , 209.66709478199618
(53, 17)
truncation:  33 +- 6
truncation:  18 +- 6
10 21 25.57219849738069 25.2697051252354
cursor at 328.0729022443222 , 217.24939160374106
(53, 17)
truncation:  33 +- 6
truncation:  18 +- 6
10 21 25.503141116714122 25.257457489812122
cursor at 328.5016766612073 , 215.05821388635877
(54, 17)
truncation:  33 +- 6
truncation:  18 +- 6
10 22 25.482581741361656 26.04004256978711
cursor at 301.1043505870325 , 182.67605659986768
(53, 17)
truncation:  33 +- 6
truncation:  19 +- 6
10 22 25.53447934843081 25.51234909786467
cursor at 319.57823986426547 , 184.3227579241541

truncation:  34 +- 6
truncation:  20 +- 6
11 22 26.634440555662913 26.232623956212965
cursor at 329.3710676514007 , 219.22432042200188
(55, 17)
truncation:  34 +- 6
truncation:  20 +- 6
11 22 26.707821308348986 26.043038180196326
cursor at 336.0082288463461 , 221.55267792149658
(56, 17)
truncation:  34 +- 6
truncation:  20 +- 6
10 25 26.73143315500593 26.35781819857853
cursor at 289.97942277636855 , 127.11244087608546
(55, 17)
truncation:  33 +- 6
truncation:  19 +- 6
11 21 26.042807563675712 25.73230587191868
cursor at 346.8865788040731 , 232.1817288819626
(54, 17)
truncation:  34 +- 6
truncation:  19 +- 6
11 21 25.95032674916466 25.428158810859884
cursor at 357.53438748271043 , 229.24733000835914
(55, 17)
truncation:  34 +- 6
truncation:  19 +- 6
11 21 26.11005324674661 25.88807075869392
cursor at 341.4334446936904 , 234.31542177492855
(55, 17)
truncation:  34 +- 6
truncation:  19 +- 6
11 22 26.354766739123686 25.789755004043595
cursor at 344.8753564516677 , 210.35032275483167
(55, 1

...

(60, 19)
truncation:  34 +- 6
truncation:  19 +- 6
9 26 27.188030020248107 31.13217907543355
cursor at 87.82626159137556 , 109.87036147857569
(60, 19)
truncation:  34 +- 6
truncation:  19 +- 6
8 26 26.844331909885845 30.448771221974297
cursor at 76.74276601594624 , 98.96488499419505
(60, 18)
truncation:  34 +- 6
truncation:  19 +- 6
7 27 26.91651841592816 31.43160472655002
cursor at 7.32624190153831 , 69.52553710374009
(59, 18)
truncation:  34 +- 6
truncation:  19 +- 6
7 27 26.712894615518938 31.09307950075762
cursor at 19.177587183593005 , 63.064592163768104
(59, 18)
truncation:  34 +- 6
truncation:  19 +- 6
7 27 26.727144126329488 31.22706531912532
cursor at 14.486911052567962 , 63.51672646527648
(59, 18)
truncation:  34 +- 6
truncation:  19 +- 6
8 26 26.898614753110802 30.684450823968877
cursor at 68.49191755218101 , 100.68726941368207
(59, 18)
truncation:  34 +- 6
truncation:  19 +- 6
7 27 26.29510529914346 29.296034512862786
cursor at 82.08988740900713 , 49.80821562926825
(59, 19)

(60, 18)
truncation:  35 +- 6
truncation:  19 +- 6
9 27 28.468647806600877 28.274573649657682
cursor at 187.86745793492776 , 118.7743111306222
(60, 19)
truncation:  35 +- 6
truncation:  19 +- 6
10 26 28.374059343451904 29.735920701313056
cursor at 171.71627395380028 , 147.50284912988144
(60, 18)
truncation:  35 +- 6
truncation:  19 +- 6
8 28 28.23579875904732 29.163805939380925
cursor at 121.72779542939975 , 79.65624241992886
(60, 19)
truncation:  35 +- 6
truncation:  20 +- 6
9 28 27.260182772610833 31.46880441915876
cursor at 76.04142880723214 , 48.70013042194684
(60, 19)
truncation:  36 +- 6
truncation:  20 +- 6
10 27 29.269016061521317 28.483876229365027
cursor at 215.54878690999465 , 144.16984552468625
(60, 19)
truncation:  35 +- 6
truncation:  20 +- 6
8 29 28.682671997258012 29.343867786528907
cursor at 115.42405508721345 , 62.1056341631159
(61, 19)
truncation:  35 +- 6
truncation:  20 +- 6
11 26 29.081802586308683 29.045790784417473
cursor at 230.88561109780267 , 169.959409289082

(65, 20)
truncation:  36 +- 6
truncation:  21 +- 6
13 28 29.26208789426811 31.441612902822982
cursor at 217.028373178479 , 112.2202039108111
(65, 20)
truncation:  36 +- 6
truncation:  21 +- 6
12 29 28.52854100627448 32.43381894865184
cursor at 147.28372810262363 , 57.215086769900566
(65, 20)
truncation:  36 +- 6
truncation:  21 +- 6
11 29 28.37735396684304 33.212931697402645
cursor at 84.99921316936559 , 52.41795040646552
(65, 20)
truncation:  35 +- 6
truncation:  21 +- 6
11 29 28.679880464886516 35.08559316002456
cursor at 19.439674621003178 , 62.0170593653067
(65, 20)
truncation:  35 +- 6
truncation:  20 +- 6
11 29 27.792539147128824 32.98865809935989
cursor at 92.85075168272715 , 33.86188602371832
(64, 20)
truncation:  36 +- 6
truncation:  22 +- 6
10 31 28.67083616588785 32.039870477379075
cursor at 91.05787030084548 , -1.7295388796357685
(64, 20)
truncation:  35 +- 6
truncation:  20 +- 6
10 29 27.639181831574252 32.33423255074135
cursor at 80.75262181842606 , 28.995887206543067
(64

11 30 37.10677016902959 23.020661422936975
cursor at 441.8178636790058 , 297.6708746721548
(62, 19)
truncation:  59 +- 6
truncation:  24 +- 6
10 29 39.4217008573575 23.750826370017588
cursor at 381.2469501384745 , 402.85300276360226
(63, 20)
truncation:  58 +- 6
truncation:  22 +- 6
11 29 39.87237058829439 23.207370814672807
cursor at 435.2814011046607 , 417.15266867615026
(63, 20)
truncation:  34 +- 6
truncation:  21 +- 6
12 28 29.119798886058593 32.67908046562733
cursor at 138.69742876474334 , 107.70540040671506
(63, 20)
truncation:  35 +- 6
truncation:  21 +- 6
12 28 29.01778969378019 30.267615182733547
cursor at 223.11981600742433 , 104.46866789628538
(63, 20)
truncation:  36 +- 6
truncation:  22 +- 6
12 28 29.925136208301158 30.93857711786478
cursor at 199.63027679880918 , 133.2586023735616
(63, 20)
truncation:  35 +- 6
truncation:  20 +- 6
11 29 27.437274556561665 31.75886848403378
cursor at 135.90414990851514 , 22.58940729498834
(63, 20)
truncation:  35 +- 6
truncation:  21 +- 6

(66, 21)
truncation:  36 +- 6
truncation:  22 +- 6
13 28 28.67267287079836 32.23201127473508
cursor at 189.3575135136556 , 93.51817592696499
(65, 21)
truncation:  35 +- 6
truncation:  22 +- 6
13 28 28.74937302855401 32.64297881161623
cursor at 174.9700534125849 , 95.95185752582779
(65, 20)
truncation:  35 +- 6
truncation:  21 +- 6
12 28 27.71821816087317 31.851427137438275
cursor at 167.6725379113376 , 63.233507257927144
(66, 21)
truncation:  35 +- 6
truncation:  21 +- 6
12 30 28.530300312753642 32.76904284173183
cursor at 135.54795835493007 , 25.541097065770188
(65, 21)
truncation:  35 +- 6
truncation:  21 +- 6
13 29 28.436659440950077 31.97217038218713
cursor at 198.45421857832184 , 54.299701960431484
(65, 20)
truncation:  35 +- 6
truncation:  21 +- 6
12 28 27.658830478226303 32.15254581111742
cursor at 157.13074929191464 , 61.34914724243376
(66, 21)
truncation:  36 +- 6
truncation:  22 +- 6
14 28 28.647642382942838 32.521823733556886
cursor at 214.22029219082788 , 92.72396324882939
